In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import unicodedata
import tensorflow_text as tf_text
import re

2023-12-11 10:28:02.783568: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 10:28:02.816613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 10:28:02.816646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 10:28:02.817366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 10:28:02.822178: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 10:28:02.822677: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Positional encoding

In [17]:
class PositionalEncoder(keras.layers.Layer):
    def __init__(self, max_seq_len, emb_dim, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if emb_dim % 2 == 1: emb_dim += 1
        k, i = np.meshgrid(np.arange(max_seq_len), np.arange(emb_dim // 2))
        pos_encoding = np.empty((1, max_seq_len, emb_dim))
        pos_encoding[0, :, ::2] = np.sin(k / 10000**(2 * i / emb_dim)).T
        pos_encoding[0, :, 1::2] = np.cos(k / 10000**(2 * i / emb_dim)).T
        self.positional_embedding = tf.constant(pos_encoding.astype(self.dtype))

    def call(self, inputs):
        shape = inputs.shape
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

## Transformer Input Layer

In [18]:
vocab_size = 10000; embed_dim = 512; max_seq_len = 500
encoder_input = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_input = keras.layers.Input(shape=[None], dtype=np.int32)

embedding_layer = keras.layers.Embedding(vocab_size, embed_dim)

encoder_embeddings = embedding_layer(encoder_input)
decoder_embeddings = embedding_layer(decoder_input)

positional_encoder = PositionalEncoder(max_seq_len, embed_dim)

encoder_in = positional_encoder(encoder_embeddings)
decoder_in = positional_encoder(decoder_embeddings)